In [21]:
!python ./MaskGIT-pytorch/Calculate_IS.py --batch-size=10 --num-codebook-vectors=16384

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
./checkpoints/transformer_epoch_30.pt
Traceback (most recent call last):
  File "./MaskGIT-pytorch/Calculate_IS.py", line 151, in <module>
    testpic = TestTransformer(args)
  File "./MaskGIT-pytorch/Calculate_IS.py", line 37, in __init__
    self.model = self.load_transformer(args).to(device=args.device)
  File "./MaskGIT-pytorch/Calculate_IS.py", line 54, in load_transformer
    sd = torch.load(args.transformer_checkpoint_path, map_location="cpu")["state_dict"]
  File "/opt/conda/lib/python3.7/site-packages/torch/serialization.py", line 608, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/opt/conda/lib/python3.7/site-packages/torch/serialization.py", line 777, in _legacy_load
    magic_number = pickle_module.load(f, **pickle_load_args)
_pickle.Un

In [19]:
!nvidia-smi

Tue Mar 29 04:53:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    67W / 149W |      0MiB / 11441MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
from omegaconf import OmegaConf
config_path = r"./taming-transformers/logs/vqgan_imagenet_f16_16384/configs/model.yaml"
config = OmegaConf.load(config_path)
config.model.params['n_embed']

16384

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import utils as vutils
from transformer import VQGANTransformer
from utils import load_data, plot_images
import sys
sys.path.insert(0, '/home/ygong2832/taming-transformers')
sys.path.insert(0, '/home/ygong2832/taming-transformers/taming')
# import model
model = VQGANTransformer(args).to(device=args.device)
model.load_state_dict(torch.load('path'))
model.eval()
# test
with tqdm(range(len(test_dataset))) as pbar:
    for i, imgs in zip(pbar, test_dataset):
        real = imgs.to(device=args.device)
        vutils.save_image(real, os.path.join("real", f"{i}.jpg"))
        logits, target = model(real)
        vutils.save_image(real, os.path.join("fake", f"{i}.jpg"))